<a href="https://colab.research.google.com/github/Sai25Hajime/PRODIGY_ML_05/blob/main/Food_Item_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [12]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os

# Check if GPU is available
print(f"TensorFlow version: {tf.__version__}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

# Set up paths
data_dir = '/content/drive/MyDrive/Datasets/food-101/images'  # Update with your actual path

# Data Augmentation and Data Generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% data for validation
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size
    class_mode='categorical',
    subset='validation'
)

# Load pre-trained MobileNetV2 model + higher level layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(101, activation='softmax')  # 101 food classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Setup callbacks for efficiency
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2),
    ModelCheckpoint('/content/drive/MyDrive/Datasets/food_recognition_model.keras', save_best_only=True)
]

# Train the model with reduced epochs, batch size, and callbacks
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=3,  # Reduced epochs
    steps_per_epoch=train_generator.samples // 16,  # Updated batch size
    validation_steps=validation_generator.samples // 16,  # Updated batch size
    callbacks=callbacks
)

# Save the trained model to Google Drive
model.save('/content/drive/MyDrive/Datasets/food_recognition_model.keras')

# Example calorie mapping (replace with actual calorie values for all 101 classes)
calorie_mapping = {
    'apple_pie': 265,
    'pizza': 285,
    # Add mappings for all 101 classes
}

# Function to predict food item and estimate calories
def predict_and_estimate_calories(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)
    predicted_food = list(train_generator.class_indices.keys())[predicted_class[0]]

    estimated_calories = calorie_mapping.get(predicted_food, "Unknown")

    print(f"Predicted Food: {predicted_food}")
    print(f"Estimated Calories: {estimated_calories} kcal")

# Example usage of the prediction function
# Replace 'path_to_test_image.jpg' with the actual path to the test image
test_image_path = '/content/drive/MyDrive/Datasets/food-101/images/apple_pie/100.jpg'  # Example image path
predict_and_estimate_calories(test_image_path, model)


TensorFlow version: 2.17.0
GPU is NOT AVAILABLE
Found 20267 images belonging to 101 classes.
Found 5066 images belonging to 101 classes.
Epoch 1/3


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1266/1266 ━━━━━━━━━━━━━━━━━━━━ 1565s 1s/step - accuracy: 0.5188 - loss: 1.6760 - val_accuracy: 0.6345 - val_loss: 1.1954
Epoch 2/3
   1/1266 ━━━━━━━━━━━━━━━━━━━━ 23:44 1s/step - accuracy: 0.6250 - loss: 1.4178

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1266/1266 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6250 - loss: 1.4178 - val_accuracy: 0.7000 - val_loss: 0.9643
Epoch 3/3
1266/1266 ━━━━━━━━━━━━━━━━━━━━ 1557s 1s/step - accuracy: 0.6813 - loss: 1.0260 - val_accuracy: 0.6697 - val_loss: 1.0839
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted Food: poutine
Estimated Calories: Unknown kcal
